# POS tagging using modified Viterbi

## Step 1: Data Preparation

In [1]:
#Importing libraries
import nltk
import random
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

print(len(nltk_data))

3914


In [3]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))

3718
196


In [4]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95539

In [5]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['In', 'one', 'feature', ',', 'called', '*', '``', 'In', 'the', 'Dumpster']

### Identifying the vocabulary
This will be helpful to check if any particular test word belongs to the vocabulary or not

In [6]:
# vocabulary
vocabulary = set(tokens)
print(len(vocabulary))

12083


### Identifying the tags used in train data
This will be useful in building TAGS MATRIX for calculating transitional probability

In [7]:
# number of tags
all_tags = [pair[1] for pair in train_tagged_words]
tags = set([pair[1] for pair in train_tagged_words])
print(len(tags))

# Tag types
print(tags)

12
{'.', 'PRON', 'NOUN', 'CONJ', 'VERB', 'DET', 'ADV', 'PRT', 'ADJ', 'NUM', 'X', 'ADP'}


VERB - verbs (all tenses and modes)
NOUN - nouns (common and proper)
PRON - pronouns
ADJ - adjectives
ADV - adverbs
ADP - adpositions (prepositions and postpositions)
CONJ - conjunctions
DET - determiners
NUM - cardinal numbers
PRT - particles or other function words
X - other: foreign words, typos, abbreviations
. - punctuation

In [8]:
from collections import Counter
tag_counts = Counter(all_tags)
tag_counts

Counter({'ADP': 9385,
         'NUM': 3326,
         'NOUN': 27393,
         '.': 11056,
         'VERB': 12893,
         'X': 6282,
         'DET': 8313,
         'PRT': 3055,
         'PRON': 2581,
         'ADV': 3014,
         'ADJ': 6092,
         'CONJ': 2149})

#### NOUN is the commonly used tag in the training set

In [9]:
tag_counts.most_common(3)

[('NOUN', 27393), ('VERB', 12893), ('.', 11056)]

## Step 2: Building the vanilla Viterbi based POS tagger

### Defining function for calculating Emission Probability

In [10]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Defining function for calculating Transition Probability

In [11]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [12]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
tags_df

,.,PRON,NOUN,CONJ,VERB,DET,ADV,PRT,ADJ,NUM,X,ADP
.,0.091172,0.064671,0.221599,0.058520,0.089454,0.175470,0.052912,0.002442,0.045134,0.079595,0.027587,0.091353
PRON,0.041069,0.008136,0.210771,0.004649,0.481209,0.010074,0.034870,0.013173,0.072065,0.007361,0.095312,0.021310
NOUN,0.239988,0.004855,0.262293,0.042967,0.147592,0.013434,0.017048,0.043916,0.012047,0.009455,0.029168,0.177235
CONJ,0.035831,0.058167,0.351792,0.000465,0.155886,0.117729,0.054444,0.005119,0.116799,0.041415,0.008376,0.053979
VERB,0.034360,0.035290,0.110913,0.005352,0.169860,0.135190,0.081362,0.031568,0.064997,0.022570,0.217327,0.091212
DET,0.017082,0.003007,0.638037,0.000361,0.039456,0.005052,0.012751,0.000241,0.206664,0.022254,0.045952,0.009142
ADV,0.135700,0.015594,0.032183,0.007299,0.342402,0.069675,0.078301,0.013603,0.131387,0.030856,0.023557,0.119443
PRT,0.041899,0.018985,0.249427,0.002291,0.401637,0.100164,0.010147,0.001964,0.084452,0.054992,0.013093,0.020949
ADJ,0.064183,0.000657,0.699934,0.016579,0.012311,0.004760,0.004924,0.011162,0.065988,0.021011,0.021011,0.077479
NUM,0.116055,0.001203,0.356284,0.012628,0.017138,0.003307,0.003307,0.024955,0.033073,0.186410,0.210162,0.035478


### Defining Vanilla Viterbi Function
The above defined functions for calculating transitional and emmision probablities will be used here

In [14]:
# Viterbi Heuristic
def Viterbi1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Step 3: Building Modified Viterbi Function version 1

The Vanilla Viterbi function does not perform well in the case of unknown words.
The unkown words will have 0 emission probability, so we discard the emission probability **in case of unknown words and consider only transitional probality to calculate the max state probablity.**

In [15]:
# Viterbi Heuristic
def Viterbi2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        
        # if word is not present in vocabulary
        if word in vocabulary:
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)
        else:
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]
                # only transition probablity is considered in case of unknown word
                p.append(transition_p)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Step 4: Building Modified Viterbi Function version 2

As Vanilla Viterbi function does not perform well in the case of unknown words, we can tag unknown words using rule based taggers. We will be using a 3 layer combined tagger for tagging unkown words.

**Top Layer - Bigram Tagger with backoff as Unigram Tagger**

**Middle Layer - Unigram tagger with backoff as Regular Expression Tagger**

**Last Layer - Regex Tagger**

The unknow word will be first tried for tagging with bigram tagger, failing upon unigram tagger will be used, failing which regex tagger will be used.

Also as we know the most commonly used tag is NOUN, for fallback purpose will be using NOUN as the default tag.

In [16]:
# creating a regular expression based tagger
regexp_tagger = nltk.RegexpTagger(
     [(r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),   # cardinal numbers
      (r'(The|the|A|a|An|an)$', 'DET'),   # articles
      (r'.*able$', 'ADJ'),                # adjectives
      (r'.*ness$', 'NOUN'),               # nouns formed from adjectives
      (r'.*ly$', 'ADV'),                  # adverbs
      (r'.*s$', 'NOUN'),                  # plural nouns
      (r'.*ing$', 'VERB'),                # gerunds
      (r'.*ed$', 'VERB'),                 # past tense verbs
      (r'.*', 'NOUN')                     # nouns (default)
])

# creating a unigram tagger with backoff as regular expression tagger
u_gram_tag=nltk.UnigramTagger(train_set,backoff=regexp_tagger)

# creating a bigram tagger with backoff as unigram tagger
b_gram_tag=nltk.BigramTagger(train_set,backoff=u_gram_tag)


# Viterbi Heuristic
def Viterbi3(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        
        # if word is not present in vocabulary
        if word in vocabulary:
            #initialise list of probability column for a given observation
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
            state.append(state_max)
        else:
            # using rule based taggers for unknown words
            _tagger = b_gram_tag.tag([word])
            state_max = _tagger[0][1]
            state.append(state_max)
        
    return list(zip(words, state))

## Step 5: Evaluating all the 3 Viterbi Taggers

### Creating test data

In [17]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# # choose random 5 sents
# rndom = [random.randint(1,len(test_set)) for x in range(50)]

# # list of sents
# test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]
# print(test_run_base)

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
# print(test_tagged_words)

### Evaluating Vanilla Viterbi

In [18]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
# print(tagged_seq)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = len(check)/len(tagged_seq)
accuracy

Time taken in seconds:  804.8175868988037


0.9102589059762507

### Evaluating Modified Viterbi version 1

In [19]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
# print(tagged_seq)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = len(check)/len(tagged_seq)
accuracy

Time taken in seconds:  731.3956158161163


0.941989488028032

### Evaluating Modified Viterbi version 2

In [20]:
start = time.time()
tagged_seq = Viterbi3(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
# print(tagged_seq)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy = len(check)/len(tagged_seq)
accuracy

Time taken in seconds:  757.7581079006195


0.9571734475374732

## Step 6: Comparing the accuracy of all the 3 taggers

Accuracy of Vanilla Viterbi = **91.02%**

Accuracy of Modified Viterbi Version 1 = **94.19%**

Accuracy of Modified Viterbi Version 2 = **95.71%**

So, it clear that Accuracy of Viterbi in tandem with Rule based taggers seems to be performing better

## Step 7: Identifying the cases which were incorrectly tagged by original POS tagger and got corrected by modified Viterbi functions

### Case 1:

<font color = blue>'Android'</font> was tagged as '.', then corrected to 'NOUN'

<font color = blue>'Google'</font> was tagged as '.', 'DET' and then corrected to 'NOUN'

In [21]:
sentence_test = 'Android is a mobile operating system developed by Google.'
words = word_tokenize(sentence_test)
tagged_seq = Viterbi1(words)
print(tagged_seq)

[('Android', '.'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', '.'), ('.', '.')]


In [22]:
tagged_seq = Viterbi2(words)
print(tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'DET'), ('.', '.')]


In [23]:
tagged_seq = Viterbi3(words)
print(tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]


### Case 2:

<font color = blue>'NASA'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'invited'</font> was tagged as '.','NOUN' and then corrected to 'VERB'

<font color = blue>'ICESAT-2'</font> was tagged as '.','DET' and then corrected to 'NOUN'

<font color = blue>'Satellite'</font> was tagged as '.' and then corrected to 'NOUN'

In [24]:
sentence_test = 'NASA invited social media users to experience the launch of ICESAT-2 Satellite.'
words = word_tokenize(sentence_test)
tagged_seq = Viterbi1(words)
print(tagged_seq)

[('NASA', '.'), ('invited', '.'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', '.'), ('Satellite', '.'), ('.', '.')]


In [25]:
tagged_seq = Viterbi2(words)
print(tagged_seq)

[('NASA', 'NOUN'), ('invited', 'NOUN'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'DET'), ('Satellite', 'NOUN'), ('.', '.')]


In [26]:
tagged_seq = Viterbi3(words)
print(tagged_seq)

[('NASA', 'NOUN'), ('invited', 'VERB'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN'), ('.', '.')]


### Case 3:

<font color = blue>'Android'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'worldwide'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'smartphones'</font> was tagged as '.','DET' and then corrected as 'NOUN'

<font color = blue>'2011'</font> was tagged as '.','DET' and then corrected to 'NUM'

<font color = blue>'2013'</font> was tagged as '.','DET' and then corrected to 'NUM'

In [27]:
sentence_test = 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.'
words = word_tokenize(sentence_test)
tagged_seq = Viterbi1(words)
print(tagged_seq)

[('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', '.'), ('.', '.')]


In [28]:
tagged_seq = Viterbi2(words)
print(tagged_seq)

[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.')]


In [29]:
tagged_seq = Viterbi3(words)
print(tagged_seq)

[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]


### Case 4:

<font color = blue>'2018'</font> was tagged as '.','NOUN' and then corrected to 'NUM'

<font color = blue>'FIFA'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'Cup'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'21st'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'tournament'</font> was tagged as '.' and then corrected to 'NOUN'

<font color = blue>'contested'</font> was tagged as '.', 'NOUN' and then corrected to 'VERB'


In [30]:
sentence_test = 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.'
words = word_tokenize(sentence_test)
tagged_seq = Viterbi1(words)
print(tagged_seq)

[('The', 'DET'), ('2018', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), ('is', 'VERB'), ('the', 'DET'), ('21st', '.'), ('FIFA', '.'), ('World', 'NOUN'), ('Cup', '.'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', '.'), ('contested', '.'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]


In [31]:
tagged_seq = Viterbi2(words)
print(tagged_seq)

[('The', 'DET'), ('2018', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'NOUN'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]


In [32]:
tagged_seq = Viterbi3(words)
print(tagged_seq)

[('The', 'DET'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]
